### 六. [Goroutine与Channel](https://juejin.im/post/5b2c49185188257494642464)
1. 基本理解  
    * `Goroutine`是Go中最基本的执行单元。事实上每一个Go程序至少有一个goroutine：主goroutine。当程序启动时，它会自动创建。 
    * `Channel` : goroutine是Go语言的基本调度单位，而`channels`则是它们之间的通信机制。操作符`<-`用来指定管道的方向，发送或接收。如果未指定方向，则为双向管道。
  
2. Channels pipeline  
   第一个goroutine是一个计数器，用于生成0、1、2、……形式的整数序列，然后通过channel将该整数序列发送给第二个goroutine；第二个goroutine是一个求平方的程序，对收到的每个整数求平方，然后将平方后的结果通过第二个channel发送给第三个goroutine；第三个goroutine是一个打印程序，打印收到的每个整数  
    ```go
    func counter(out chan<- int, number int) {
        for x := 0; x < number; x++ {
            out <- x
        }
        close(out)
    }

    func squarer(out chan<- int, in <-chan int) {
        for v := range in {
            out <- v * v
        }
        close(out)
    }

    func printer(in <-chan int) {
        for v := range in {
            fmt.Print(v,",")
        }
    }

    func main() {
        naturals := make(chan int)
        squares := make(chan int)

        go counter(naturals,10)
        go squarer(squares, naturals)
        printer(squares)
    }

    ```

3. `range` 和 `close`   
    * 发送者可通过`close(chann)`关闭一个信道来表示没有需要发送的值了。
    * 接收者可以通过为接收表达式分配第二个参数来测试信道是否被关闭：若没有值可以接收且信道已被关闭, 则在执行`v, ok := <-ch`时, ok变量会被设置为false。
    * 循环`for i := range c`会不断从信道接收值，直到它被关闭。  
      **[注意]:** 只有发送者才能关闭信道，而接收者不能。向一个已经关闭的信道发送数据会引发程序恐慌（panic）。   
      **[还要注意]:** 信道与文件不同，通常情况下无需关闭它们。只有在必须告诉接收者不再有需要发送的值时才有必要关闭，例如终止一个 range 循环。
    ```go
    func fibonacci(n int, c chan int){
        x,y := 0,1
        for i:=0; i<n; i++{
            // 将x发送到信道, 也意味着这个方法会在一个新的goroutine中执行
            c <- x
            x,y = y,x+y
        }
        close(c)     // 由发送者关闭信道
    }

    func main(){
        loop := 10
        c := make(chan int)
        go fibonacci(loop,c)  // 创建goroutine
        for i:= range c{      // i为从信道接受的值
            fmt.Print(i,",")  // 0,1,1,2,3,5,8,13,21,34,
        }
    }    
    ```

4. `select{case}`
    * `case`语句块是`select`可选择的执行分支. 他会随机选择一个不会产生阻塞的分支进行处理.如果所有的`case`分支都会阻塞, 就会选择走`case default`分支进行; 如果没有default分支切所有其他分支均阻塞, 则`select`语句阻塞
    ```go
    func fibonacci(c chan int, quitChan chan int){
        x,y := 0,1
        for {
            select {
            case c<-x: // 将x发送到信道
                x,y = y,x+y
            case <-quitChan:
                fmt.Println("quit fibonacci")
                return
            }
        }
    }

    func main(){
        c := make(chan int)
        quitChan := make(chan int)
        loop := 10
        go func() {
            for i := 0; i < loop; i++ {
                fmt.Print(<-c,",")
            }
            quitChan <- 0   // 循环得到10次结果后, 发送退出信号
        }()
        fibonacci(c,quitChan)  // 此时上面的goroutine中的select才能继续进行
        
        // 0,1,1,2,3,5,8,13,21,34,quit fibonacci
    }
    ```

5. `sync.Mutex`   
   我们已经看到信道非常适合在各个Goroutine间进行通信。但是如果我们并不需要通信呢？比如只是想保证每次只有一个Goroutine能够访问一个共享的变量，从而避免冲突？
   这里涉及的概念叫做`互斥(mutual exclusion)`，我们通常使用`互斥锁（Mutex)`这一数据结构来提供这种机制。  
   Go 标准库中提供了`sync.Mutex`互斥锁类型及其两个方法：
    * Lock
    * Unlock  

    我们可以通过在代码前调用`Lock`方法，在代码后调用`Unlock`方法来保证一段代码的互斥执行。参见 Inc 方法。  
    我们也可以用`defer`语句来保证互斥锁一定会被解锁。参见 Value 方法。
    ```go
    type SafeCounter struct {
        v map[string]int
        mux sync.Mutex
    }
    /** 该方法在goroutine中运行*/
    func(counter *SafeCounter) Inc (key string){
        counter.mux.Lock()
        counter.v[key]++
        counter.mux.Unlock()
    }
    /** 该方法在goroutine中运行*/
    func (counter *SafeCounter) Value (key string) int{
        counter.mux.Lock()
        defer counter.mux.Unlock()
        return counter.v[key]
    }

    func main(){
        c := SafeCounter{v:make(map[string] int)}
        for i := 0; i < 1000; i++ {
            go c.Inc("somekey")
        }
        time.Sleep(time.Second)
        fmt.Println(c.Value("somekey"))
    }    
    ```
